In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory
import utils
import numpy as np
from google.colab import drive

drive.mount('/content/drive/')

data_path = "/content/drive/MyDrive/Aerial_Landscapes/"
img_size = 224
batch_size = 64
epochs = 10

# data augmentation
def data_augmentation(image, label):
    image = tf.image.random_contrast(image, 0.2, 0.5)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_crop(image, size=(200, 200, 3))
    image = tf.image.resize(image, [224, 224])
    return image, label

# load data set and split into train and validation
dataset = image_dataset_from_directory(
    data_path,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode='int',
    shuffle=True,
    seed=42,
    validation_split = 0.2,
    subset="both"
)

# seperate train and validation
dataset_train = dataset[0]
dataset_val = dataset[1]

# Apply data augmentation to training dataset
dataset_train = (
    dataset_train
    .unbatch()
    .map(data_augmentation)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

print("Data loaded ")
print()

print("model:EfficientNetB0 with pretrained weights")
# load model with pre-trained weights
base_model = EfficientNetB0(include_top=False, weights = "imagenet", input_shape=(img_size, img_size, 3))
# custom top layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(15, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# compile and run
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(dataset_train, validation_data=dataset_val, epochs=epochs)

# form both predition and validation into label lists
y_true = []
y_pred = []
for images, labels in dataset_val:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# pass thru evalaution function
utils.evaluate_performance(y_true, y_pred, model_name="EfficientNetB0 with pretrained weights")

#evaluate second model training from ground up with default top layer
print()
print()
print("model:EfficientNetB0 with default top layer and no pre-trained weights")

# load model without pre-trained weights
model = EfficientNetB0(include_top=True, weights = None, classes = 15, input_shape=(img_size, img_size, 3))
# compile and run
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(dataset_train, validation_data=dataset_val, epochs=epochs)

# form both predition and validation into label lists
y_true = []
y_pred = []
for images, labels in dataset_val:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# pass thru evalaution function
utils.evaluate_performance(y_true, y_pred, model_name="EfficientNetB0 with default top layer and no pre-trained weights")


Mounted at /content/drive/
Found 12090 files belonging to 15 classes.
Using 9672 files for training.
Using 2418 files for validation.
Data loaded 

model:EfficientNetB0 with pretrained weights
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
    152/Unknown 655s 4s/step - accuracy: 0.7667 - loss: 0.8205

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


152/152 ━━━━━━━━━━━━━━━━━━━━ 804s 5s/step - accuracy: 0.7674 - loss: 0.8182 - val_accuracy: 0.9107 - val_loss: 0.3074
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.9390 - loss: 0.1912 - val_accuracy: 0.8706 - val_loss: 0.4555
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.9578 - loss: 0.1331 - val_accuracy: 0.8747 - val_loss: 0.4743
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.9639 - loss: 0.1132 - val_accuracy: 0.8197 - val_loss: 0.6366
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.9689 - loss: 0.0989 - val_accuracy: 0.8759 - val_loss: 0.4490
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.9709 - loss: 0.0958 - val_accuracy: 0.8015 - val_loss: 0.8190
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.9756 - loss: 0.0840 - val_accuracy: 0.8883 - val_loss: 0.4072
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.9834 - loss: 0.0480 - val_a

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


152/152 ━━━━━━━━━━━━━━━━━━━━ 134s 445ms/step - accuracy: 0.2127 - loss: 2.9312 - val_accuracy: 0.0720 - val_loss: 2.9804
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.3769 - loss: 1.8782 - val_accuracy: 0.0724 - val_loss: 3.3570
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.4451 - loss: 1.7499 - val_accuracy: 0.3871 - val_loss: 1.9052
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.5303 - loss: 1.4249 - val_accuracy: 0.3586 - val_loss: 2.1613
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.5321 - loss: 1.4476 - val_accuracy: 0.2605 - val_loss: 3.4164
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.6039 - loss: 1.2107 - val_accuracy: 0.3164 - val_loss: 2.8932
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.6351 - loss: 1.1088 - val_accuracy: 0.2531 - val_loss: 3.1081
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.6631 - loss: 1.0312 - va